# Generating Datasets

In [1]:
import cv2 as cv
import os
import numpy as np
from PIL import Image

In [2]:
if not os.path.exists("faces"):
    os.mkdir("faces")
cap = cv.VideoCapture(0)

face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
face_id = input('Enter your Id: ')

count = 0

while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.5, 5)

    for (x,y,w,h) in faces:
        cv.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

        count += 1
        face_image = gray[y:y+h, x:x+w]
        cv.imwrite(f'faces/{face_id}_{count}.jpg', face_image)

    
    cv.putText(frame, str(count), (frame.shape[1] // 2, frame.shape[0] - 50), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv.imshow('Captured Image', frame)
        
    if count >= 400:
        break

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
cv.waitKey(1)

Enter your Id:  4


2025-02-19 22:10:45.616 Python[3604:126184] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 22:10:45.616 Python[3604:126184] +[IMKInputSession subclass]: chose IMKInputSession_Modern


-1

# Training Datasets

In [3]:
def data_classifier(faces_dir):
    # Get all image paths from the faces_dir
    path = [os.path.join(faces_dir, f) for f in os.listdir(faces_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    face = []
    ids = []

    for image in path:
        img = cv.imread(image, cv.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to load image: {image}")
            continue
        
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[-1].split('_')[0])

        face.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Training classifier
    clf = cv.face.LBPHFaceRecognizer_create()
    clf.train(face, ids)
    clf.write("trained_classifier.xml")

    print("Training complete. Classifier saved as 'trained_classifier.xml'.")

data_classifier("faces")


Training complete. Classifier saved as 'trained_classifier.xml'.


# Face Recognizer


In [8]:
def draw_boundary(img, classifier, scalefactor, minNeighbors, color, text, clf):
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    face = classifier.detectMultiScale(gray_img, scalefactor, minNeighbors)

    for (x, y, w, h) in face:
        cv.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])

        confidence = int(100 * (1 - pred / 300))

        if confidence > 85:
            if id == 2:
                cv.putText(img, "Prawal", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            if id == 3:
                cv.putText(img, "Ayudh", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            if id == 4:
                cv.putText(img, "Dinesh Bajgain", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            if id == 6:
                cv.putText(img, "Dipti", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            if id == 7:
                cv.putText(img, "ILU", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            if id == 15:
                cv.putText(img, "Roshan", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            if id == 17:
                cv.putText(img, "SRK", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        else:
            cv.putText(img, "UNKNOWN", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA)

    return img  

face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
clf = cv.face.LBPHFaceRecognizer_create()
clf.read("trained_classifier.xml")

cap = cv.VideoCapture(0) 

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, img = cap.read()

    img = draw_boundary(img, face_cascade, 1.5, 5, (255, 255, 255), "Name", clf)
    cv.imshow('Detected face', img)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
cv.waitKey(1)

-1